In [17]:
import os
import pandas as pd

In [18]:
from io import StringIO
from dotenv import load_dotenv
from azure.storage.blob import BlobServiceClient

In [19]:
service_client = BlobServiceClient.from_connection_string(os.getenv("AZURE_CONNECTION_STRING"))
container = service_client.get_container_client(os.getenv("CONTAINER_NAME"))

In [46]:
def clean_add_features(df: pd.DataFrame, source: str):

    df.insert(0, "Source", source)
    df["Volume"] = df["Volume"].str.replace(',','').astype(int)
    df["Date"] = pd.to_datetime(df["Date"], format='%m/%d/%Y')

    # Dates
    df["Month"] = df["Date"].dt.month
    df["Day"] = df["Date"].dt.day
    df["Year"] = df["Date"].dt.year
    df["DayOfWeek"] = df["Date"].dt.dayofweek

    df["MA_5"] = df["Close"].rolling(5).mean()
    df["MA_20"] = df["Close"].rolling(20).mean()
    df["EMA_5"] = df["Close"].ewm(span=5, adjust=False).mean()
    df["EMA_20"] = df["Close"].ewm(span=20, adjust=False).mean()
    df["STD_5"] = df["Close"].rolling(5).mean()
    df["STD_20"] = df["Close"].rolling(20).mean()

    df["DailyReturn"] = df["Close"].pct_change()
    df["Volatility"] = df["DailyReturn"].rolling(20).std()
    df["PriceChange"] = df["Close"] - df["Open"]
 
    return df

In [49]:
def save_silver(data):
    blob_name = f"bronze/etf_silver.csv"
    blob_client = container.get_blob_client(blob=blob_name)
    
    # Convert list to DataFrame
    df = pd.DataFrame(data)

    # Save to CSV file
    df.to_csv("output.csv", index=False)
    
    #data = data.to_csv("output.csv", index=False)
    blob_client.upload_blob(data, overwrite=True)

In [50]:

for blob in container.list_blobs(name_starts_with="bronze/bdo.csv"):
    if blob.name.endswith('.csv'):
        blob_client = container.get_blob_client(blob.name)
        content = blob_client.download_blob().readall().decode('utf-8')

        file = pd.read_csv(StringIO(content))
        final = clean_add_features(file, os.path.basename(blob.name).replace('.csv',''))

#staging
save_silver(final)

ClientAuthenticationError: Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:61d77a8d-601e-000f-3459-ffa903000000
Time:2025-07-28T00:52:40.8171387Z
ErrorCode:AuthenticationFailed
authenticationerrordetail:The MAC signature found in the HTTP request 'n6JA9S37fPnKTPmlchs6UVvOShjUx7RMxN587EJSbt0=' is not the same as any computed signature. Server used following string to sign: 'PUT


116

application/octet-stream






x-ms-blob-type:BlockBlob
x-ms-client-request-id:2a8c1c83-6b4d-11f0-93d9-1068380ac032
x-ms-date:Mon, 28 Jul 2025 00:52:41 GMT
x-ms-version:2025-05-05
/etfstocksmarketdata/data-container/bronze/etf_silver.csv'.
Content: <?xml version="1.0" encoding="utf-8"?><Error><Code>AuthenticationFailed</Code><Message>Server failed to authenticate the request. Make sure the value of Authorization header is formed correctly including the signature.
RequestId:61d77a8d-601e-000f-3459-ffa903000000
Time

In [44]:
final

,Source,Date,Open,High,Low,Close,Volume,Month,Day,Year,DayOfWeek,MA_5,MA_20,EMA_5,EMA_20,STD_5,STD_20,DailyReturn,Volatility,PriceChange
0,bdo,2025-07-24,152.2,153.9,152.0,152.2,1307730,7,24,2025,3,NaN,NaN,152.200000,152.200000,NaN,NaN,NaN,NaN,0.0
1,bdo,2025-07-22,149.9,149.9,148.2,149.9,2438500,7,22,2025,1,NaN,NaN,151.433333,151.980952,NaN,NaN,-0.015112,NaN,0.0
2,bdo,2025-07-21,150.9,151.0,148.9,151.0,1228600,7,21,2025,0,NaN,NaN,151.288889,151.887528,NaN,NaN,0.007338,NaN,0.1
3,bdo,2025-07-17,147.9,149.8,147.5,148.6,2047490,7,17,2025,3,NaN,NaN,150.392593,151.574430,NaN,NaN,-0.015894,NaN,0.7
4,bdo,2025-07-16,150.0,151.1,147.2,151.4,3485930,7,16,2025,2,150.62,NaN,150.728395,151.557818,150.62,NaN,0.018843,NaN,1.4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321,bdo,2024-03-22,154.5,155.1,153.5,154.2,3800680,3,22,2024,4,154.40,150.530,153.685807,151.001864,154.40,150.530,0.005871,0.016249,-0.3
322,bdo,2024-03-21,154.0,155.6,154.0,154.5,3289060,3,21,2024,3,153.50,151.050,153.957204,151.335020,153.50,151.050,0.001946,0.015950,0.5
323,bdo,2024-03-20,153.4,154.8,151.8,154.0,3733010,3,20,2024,2,153.40,151.570,153.971470,151.588828,153.40,151.570,-0.003236,0.015944,0.6
324,bdo,2024-03-19,152.9,154.0,151.3,153.5,2238910,3,19,2024,1,153.90,151.905,153.814313,151.770844,153.90,151.905,-0.003247,0.015384,0.6
